In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import IFrame, display
InteractiveShell.ast_node_interactivity = "all"

# 🔌Hybrid additive playground

Just a place to get linkers running.

In [2]:
from src import locations as loc
from src.data import utils as du
from src.data.star import Star
from src.data.datasets import Dataset
from src.data.probabilities import Probabilities
from src.data.clusters import Clusters
from src.link.splink_linker import SplinkLinker
from src.config import link_pipeline, stopwords
from src.features.clean_complex import clean_comp_names

import splink.duckdb.comparison_library as cl
import splink.duckdb.comparison_template_library as ctl

import uuid
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
import os

dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

DEBUG:git.cmd:Popen(['git', 'version'], cwd=/home/jovyan/company-matching/notebooks, universal_newlines=False, shell=None, istream=None)
DEBUG:git.cmd:Popen(['git', 'version'], cwd=/home/jovyan/company-matching/notebooks, universal_newlines=False, shell=None, istream=None)


True

## Setup

In [3]:
star = Star(
    schema = os.getenv("SCHEMA"),
    table = os.getenv("STAR_TABLE")
)
probabilities = Probabilities(
    schema = os.getenv("SCHEMA"),
    table = os.getenv("PROBABILITIES_TABLE"),
    star = star
)
clusters = Clusters(
    schema = os.getenv("SCHEMA"),
    table = os.getenv("CLUSTERS_TABLE"),
    star = star
)

In [4]:
# probabilities.create(overwrite=True)
# clusters.create(dim=1970, overwrite=False)

## Splink

TODO:

* Write the `linker.link` method
* Run it
* Load it into clusters
* Add `cluster_select`s to the `link_pipeline` in config

At that point I think we've got enough infra to MR the whole of this, linker, data and all. Not a pretty MR, a lot to chew, sorry reviewer.

In [5]:
# '"hmrc"."trade__exporters"': {
#     "fact": '"hmrc"."trade__exporters"',
#     "key_fields": ["company_name", "address", "postcode"],
#     "dim": f'"{os.getenv("SCHEMA")}"."hmrc_trade__exporters__dim"',
#     "n": 3,
#     "experiment": "cm_hmrc-trade-exporters",
# }

In [6]:
cl_x_exp = SplinkLinker(
    dataset = Dataset(
        star_id=54717,
        star=star
    ), 
    probabilities=probabilities, 
    clusters=clusters, 
    n=2
)
cl_x_exp.get_data(
    cluster_select={
        '"companieshouse"."companies"': [
            "company_name as company_name",
            "postcode as postcode"
        ]
    },
    dim_select=[
        "id",
        "company_name",
        "postcode"
    ]
)

/opt/conda/envs/company_matching/lib/python3.9/site-packages/pandas/io/sql.py:1410: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  meta = MetaData(self.connectable, schema=schema)


In [8]:
cl_x_exp=SplinkLinker.load(
    path=Path(loc.DATA_SUBDIR['raw'], 'ch_x_exp.pickle')
)

In [9]:
cl_x_exp.cluster_raw.head()
cl_x_exp.dim_raw.head()

,cluster,company_name,postcode
0,9834ef62-a2fc-430e-935e-b77f96ccc3d5,"""ITRESIDENT""",E2 0LT
1,53e4217f-5649-47f6-8d20-4bcde95b7386,$AVE LTD,HG1 1ND
2,4d6c65be-7203-4b97-a8f8-b5af44e2b98a,'OUT AND ABOUT' WITH HIGHWAY FARM C.I.C.,PL25 4TR
3,77cd1f1f-a235-4dfb-ae2d-4ad5db3cf753,'YOUR HAIR' BY TJ LTD,FY8 1QG
4,346c616d-049e-4508-b6fe-efab8f6a8f46,(THE) COMMERCIAL COMPANY OF SALONICA LTD,W1S 2FB


,id,company_name,postcode
0,2749503,001-01062021 LTD,TF1 7ET
1,2828512,001 SECONDS LIMITED,TW13 5LR
2,349156,0044 LTD,CV32 6SN
3,1432833,0044 LTD,YO30 5PB
4,1413256,01DIRECT LTD,NW4 4QE


Watched this crawl up to 30GB in memory.

This cleaning function needs refactoring immediately.

In [7]:
cl_x_exp._clean_data(
    cluster_pipeline={
        "clean_comp_names": {
            "function": clean_comp_names,
            "arguments": {
                "primary_col": "company_name",
                "secondary_col": None,
                "stopwords": stopwords,
            },
        }
    },
    dim_pipeline={
        "clean_comp_names": {
            "function": clean_comp_names,
            "arguments": {
                "primary_col": "company_name",
                "secondary_col": None,
                "stopwords": stopwords,
            },
        }
    }
)

In [11]:
cl_x_exp.cluster_processed.head()
cl_x_exp.dim_processed.head()

,cluster,company_name,postcode
0,9834ef62-a2fc-430e-935e-b77f96ccc3d5,itresident,E2 0LT
1,53e4217f-5649-47f6-8d20-4bcde95b7386,ave,HG1 1ND
2,4d6c65be-7203-4b97-a8f8-b5af44e2b98a,out about with highway farm cic,PL25 4TR
3,77cd1f1f-a235-4dfb-ae2d-4ad5db3cf753,your hair by tj,FY8 1QG
4,346c616d-049e-4508-b6fe-efab8f6a8f46,commercial salonica,W1S 2FB


,id,company_name,postcode
0,2749503,001 01062021,TF1 7ET
1,2828512,001 seconds,TW13 5LR
2,349156,0044,CV32 6SN
3,1432833,0044,YO30 5PB
4,1413256,01direct,NW4 4QE


In [8]:
cl_x_exp.save(path=Path(loc.DATA_SUBDIR['raw'], 'ch_x_exp.pickle'))

In [9]:
cl_x_exp._create_linker(
    n=1,
    linker_settings={
        "link_type": "link_only",
        "unique_id_column_name": "id",
        "retain_matching_columns": False,
        "retain_intermediate_calculation_columns": False,
        "blocking_rules_to_generate_predictions": [
            """
                (l.company_name = r.company_name)
                and (
                    l.company_name <> ''
                    and r.company_name <> ''
                )
            """,
            """
                (l.postcode = r.postcode)
                and (
                    l.postcode <> ''
                    and r.postcode <> ''
                )
            """,
        ],
        "comparisons": [
            cl.jaro_winkler_at_thresholds(
                "company_name", [0.9, 0.6], term_frequency_adjustments=True
            ),
            ctl.postcode_comparison("postcode"),
        ],
    }
)

ParserException: Parser Error: syntax error at or near "1"
LINE 1: select * from 1 limit 1
                      ^

In [24]:
cl_x_exp._train_linker(
    train_pipeline={
        "estimate_probability_two_random_records_match": {
            "function": "estimate_probability_two_random_records_match",
            "arguments": {
                "deterministic_matching_rules": """
                    l.name_unusual_tokens = r.name_unusual_tokens
                """,
                "recall": 0.7,
            },
        },
        "estimate_u_using_random_sampling": {
            "function": "estimate_u_using_random_sampling",
            "arguments": {"max_pairs": 1e6},
        },
        "estimate_parameters_using_expectation_maximisation": {
            "function": "estimate_parameters_using_expectation_maximisation",
            "arguments": {
                "blocking_rule": """
                    l.name_unusual_tokens = r.name_unusual_tokens
                """
            },
        },
    }
)

CatalogException: Catalog Error: Table with name None does not exist!
Did you mean "temp.information_schema.columns"?

In [ ]:
cl_x_exp.link(threshold=0.7, log_output=True)